`wu_fuzzy_NN_training.ipynb`
> trying to make the code more compact

In [2]:
import pandas as pd, tensorflow as tf,  matplotlib.pyplot as plt, sys
from sklearn.preprocessing import MinMaxScaler

from matplotlib.ticker import MultipleLocator, FormatStrFormatter, FuncFormatter
from plot_styles import set_plotstyles
set_plotstyles()
%config InlineBackend.figure_format = 'retina'

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
#import data
data_path_1 = 'data/wu_fuzzy_rand_tau_rand_phase_more_noise.csv'
data_df_1 = pd.read_csv(data_path_1, dtype = float)

# split random 80% / 10%
training_data_df_1 = data_df_1.sample(frac = 0.8)
testing_data_df_1 = data_df_1.loc[~data_df_1.index.isin(training_data_df_1.index)]

# separate input features from output
X_training_1 = training_data_df_1.drop('slope', axis = 1).values
Y_training_1 = training_data_df_1[['slope']].values

X_testing_1 = testing_data_df_1.drop('slope', axis = 1).values
Y_testing_1 = testing_data_df_1[['slope']].values

# setup data scalers
X_scaler_1 = MinMaxScaler(feature_range = (0,1))
Y_scaler_1 = MinMaxScaler(feature_range = (0,1))

# scale data
X_scaled_training_1 = X_scaler_1.fit_transform(X_training_1)
Y_scaled_training_1 = Y_scaler_1.fit_transform(Y_training_1)

X_scaled_testing_1 = X_scaler_1.transform(X_testing_1)
Y_scaled_testing_1 = Y_scaler_1.transform(Y_testing_1)

In [4]:
number_of_inputs = X_scaled_training_1.shape[1]
number_of_outputs = Y_scaled_training_1.shape[1]

In [5]:
# def formula_layer_nodes(num_layers, num_inputs, num_ouputs):
#     r = (number_of_inputs/number_of_outputs)**(1/(num_layers + 1))
    
#     return [int(number_of_outputs * r**(n - 1)) for n in range(num_layers + 2)[2:][::-1]]

In [6]:
layer_nodes = [115,280,450,290,205,102,50]

In [7]:
learning_rate = 0.0015
training_epochs = 2000

In [8]:
run_code = 'seven_layers_LARGE_'
logdir = run_code + '_'.join([str(nodes) for nodes in layer_nodes])
logdir

'seven_layers_LARGE_115_280_450_290_205_102_50'

In [9]:
tf.reset_default_graph()

In [10]:
# Input Layer
with tf.variable_scope('input'):
    X  = tf.placeholder(tf.float32, shape = (None, number_of_inputs))

# Layer 1
with tf.variable_scope('layer_1'):

    biases = tf.get_variable(name = "biases1",
                             shape = [layer_nodes[0]],
                             initializer = tf.zeros_initializer())

    weights = tf.get_variable(name = "weights1",
                              shape  = [number_of_inputs, layer_nodes[0]],
                         initializer = tf.contrib.layers.xavier_initializer())


    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)

# Layer 2
with tf.variable_scope('layer_2'):

    biases = tf.get_variable(name = "biases2",
                             shape = [layer_nodes[1]],
                             initializer = tf.zeros_initializer())

    weights = tf.get_variable(name = "weights2",
                              shape  = [layer_nodes[0], layer_nodes[1]],
                         initializer = tf.contrib.layers.xavier_initializer())


    layer_2_output = tf.nn.leaky_relu(tf.matmul(layer_1_output, weights) + biases)

# Layer 3
with tf.variable_scope('layer_3'):

    biases = tf.get_variable(name = "biases3",
                             shape = [layer_nodes[2]],
                             initializer = tf.zeros_initializer())

    weights = tf.get_variable(name = "weights3",
                              shape  = [layer_nodes[1], layer_nodes[2]],
                         initializer = tf.contrib.layers.xavier_initializer())


    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)

 # Layer 4
with tf.variable_scope('layer_4'):

    biases = tf.get_variable(name = "biases4",
                             shape = [layer_nodes[3]],
                             initializer = tf.zeros_initializer())

    weights = tf.get_variable(name = "weights4",
                              shape  = [layer_nodes[2], layer_nodes[3]],
                         initializer = tf.contrib.layers.xavier_initializer())


    layer_4_output = tf.nn.relu(tf.matmul(layer_3_output, weights) + biases)
    
 # Layer 3
with tf.variable_scope('layer_5'):

    biases = tf.get_variable(name = "biases5",
                             shape = [layer_nodes[4]],
                             initializer = tf.zeros_initializer())

    weights = tf.get_variable(name = "weights5",
                              shape  = [layer_nodes[3], layer_nodes[4]],
                         initializer = tf.contrib.layers.xavier_initializer())


    layer_5_output = tf.nn.relu(tf.matmul(layer_4_output, weights) + biases)
    
# Layer 6
with tf.variable_scope('layer_6'):

    biases = tf.get_variable(name = "biases6",
                             shape = [layer_nodes[5]],
                             initializer = tf.zeros_initializer())

    weights = tf.get_variable(name = "weights6",
                              shape  = [layer_nodes[4], layer_nodes[5]],
                         initializer = tf.contrib.layers.xavier_initializer())


    layer_6_output = tf.nn.relu(tf.matmul(layer_5_output, weights) + biases)

# Layer 7
with tf.variable_scope('layer_7'):

    biases = tf.get_variable(name = "biases7",
                             shape = [layer_nodes[6]],
                             initializer = tf.zeros_initializer())

    weights = tf.get_variable(name = "weights7",
                              shape  = [layer_nodes[5], layer_nodes[6]],
                         initializer = tf.contrib.layers.xavier_initializer())


    layer_7_output = tf.nn.relu(tf.matmul(layer_6_output, weights) + biases)
# Output layer

with tf.variable_scope('output'):

    biases = tf.get_variable(name = "biases_out",
                             shape = [number_of_outputs],
                             initializer = tf.zeros_initializer())

    weights = tf.get_variable(name = "weights_out",
                              shape  = [layer_nodes[6], number_of_outputs],
                         initializer = tf.contrib.layers.xavier_initializer())


    prediction = tf.nn.relu(tf.matmul(layer_7_output, weights) + biases)

with tf.variable_scope('cost'):

    Y = tf.placeholder(tf.float32, shape = (None, 1))
    cost = tf.reduce_mean(tf.squared_difference(prediction, Y))


with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate, epsilon=1e-08).minimize(cost)

with tf.variable_scope('logging'):
    tf.summary.scalar('current_cost', cost)
    summary = tf.summary.merge_all()
    

In [11]:
saver = tf.train.Saver()

In [ ]:
with tf.Session() as session:

    session.run(tf.global_variables_initializer())

    training_writer = tf.summary.FileWriter('logs/' + logdir + '/training', session.graph)
    testing_writer = tf.summary.FileWriter('logs/' + logdir + '/testing', session.graph)

    for epoch in range(training_epochs):

        # Feed in the training data and do one stepf NN training
        session.run(optimizer,
                    feed_dict = {X: X_scaled_training_1, Y: Y_scaled_training_1})

        training_feed = {X: X_scaled_training_1, Y: Y_scaled_training_1}
        training_cost, training_summary = session.run([cost, summary],
                                                feed_dict = training_feed)

        testing_feed =  {X: X_scaled_testing_1, Y: Y_scaled_testing_1}
        testing_cost, testing_summary = session.run([cost, summary],
                                                     feed_dict = testing_feed)
        sys.stdout.write('\r epoch: {:4.0f} testing_cost: {:2.10f}'.format(epoch, testing_cost))
        
        training_writer.add_summary(training_summary, epoch)
        testing_writer.add_summary(testing_summary, epoch)
        testing_writer.flush()
        training_writer.flush()

    model_save_location = 'logs/' + logdir + '_model' + '/trained_model.ckpt'
    save_path = saver.save(session, model_save_location)

 epoch:  358 testing_cost: 0.0698450878

In [ ]:
with tf.Session() as session:
    saver.restore(session,'logs/seven_layers_LARGE_115_280_450_290_205_102_50_model/trained_model.ckpt')
    
    prediction_scaled = session.run(prediction, feed_dict = {X: X_scaled_testing})
    
    # the predictions but in the original units
    prediction_unscaled = Y_scaler.inverse_transform(prediction_scaled)

In [ ]:
sorting_idx = np.argsort(Y_testing_1.T[0])

fig, ax = plt.subplots(figsize = (8.5, 3), dpi = 100)

ax.set_title('NN predictions vs actual, final testing cost ?')
ax.plot(prediction_unscaled.T[0][sorting_idx],
        linestyle = 'none', marker = '.', markersize = 1,
        color = 'darkblue', label = 'Prediction')

ax.plot(Y_testing_1.T[0][sorting_idx],
        linestyle = 'none', marker = '.', markersize = 1, alpha = 0.75, 
        color = '#E50000', label = 'Actual')

ax.set_ylabel(r'$\tau$ [ ns ]')
ax.set_xlabel(r'$\texttt{Y\_testing}$ sample number')
ax.legend( loc = 'best')
plt.show()

In [ ]:
NN_SCORE = sum((Y_scaled_testing_1.T[0] - prediction_scaled.T[0])**2)
NN_SCORE

In [ ]:
np.std(abs(Y_testing_1.T[0] - prediction_unscaled.T[0]))

In [ ]:
np.mean(abs(Y_testing_1.T[0] - prediction_unscaled.T[0]))